In [3]:
import pandas as pd
pd.read_csv('VoosAzul.csv').to_json('VoosAzul.json')

In [4]:
from math import radians, cos, sin, asin, sqrt

# Formula de Haversine
def haversine( a, b ):
    # Raio da Terra em Km
    r = 6371

    # Converte coordenadas de graus para radianos
    lon1, lat1, lon2, lat2 = map(radians, [ a['longitude'], a['latitude'], b['longitude'], b['latitude'] ] )

    # Formula de Haversine
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    hav = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    d = 2 * r * asin( sqrt(hav) )

    return d


AdalbertoMendesDaSilva = {'latitude': -25.001815, 'longitude': -53.501918 }
Viracopos = {'latitude': -23.008205, 'longitude': -47.1375685}

print("Adalberto x Viracopos: " + str( haversine( AdalbertoMendesDaSilva, Viracopos) ) + " Km")

Adalberto x Viracopos: 683.3258154870508 Km


In [45]:
graph = {}
graph['s'] = {}
graph['s'] = {'a': 2}

# templist = graph['s']
# templist.extend({'b': 2})
# graph['s'] = templist

graph['s']['b'] = 3

graph

{'s': {'a': 2, 'b': 3}}

In [79]:
import json

with open('VoosAzul.json') as json_file:  
    data = json.load(json_file)
    
# print(data)
listaDeAdjacencia = {}

# Montando lista de adjacência
for (key, aeroporto) in data['Aeroporto.Origem'].items():
    if not listaDeAdjacencia.get(aeroporto):
        listaDeAdjacencia[aeroporto] = {}
    
    cordOrigem = {'latitude': data['LatOrig'][key], 'longitude': data['LongOrig'][key]}
    cordDestino = {'latitude': data['LatDest'][key], 'longitude': data['LongDest'][key]}
    
#     listaDeAdjacencia[aeroporto].append([data['Aeroporto.Destino'][key], haversine(cordOrigem, cordDestino)])
    listaDeAdjacencia[aeroporto][data['Aeroporto.Destino'][key]] = haversine(cordOrigem, cordDestino)
    
        
# Salvando lista de adjacência
with open('listaAdjacência.json', 'w') as json_file:  
    json.dump(listaDeAdjacencia, json_file)
    
print(listaDeAdjacencia)           

NameError: name 'haversine' is not defined

In [86]:
def BFS(grafo, aeroporto_origem, aeroporto_destino):
    fila = []
    visitados = []
    largura = {}
    ctdLargura = 1
    nivel = {}
    pai = {}
    
    fila.append(aeroporto_origem)
    largura[aeroporto_origem] = ctdLargura
    nivel[aeroporto_origem] = 1
    pai[aeroporto_origem] = None
    
    while len(fila):
        vertice = fila.pop(0) 
        
        for vizinho in grafo.get(vertice):
            if not largura.get(vizinho):
                fila.append(vizinho)
                ctdLargura += 1
                largura[vizinho] = ctdLargura
                pai[vizinho] = vertice
                nivel[vizinho] = nivel[vertice] + 1
                if vizinho == aeroporto_destino:
                    return largura, nivel, pai
    return 0

def imprime_menor_caminho(pai, aeroporto_origem, aeroporto_destino):
    menor_caminho = []
    while aeroporto_destino != aeroporto_origem:
        menor_caminho.insert(0, aeroporto_destino)
        aeroporto_destino = pai[aeroporto_destino]
        
    menor_caminho.insert(0, aeroporto_origem)
    return menor_caminho
    

In [87]:
largura, nivel, pai = BFS(listaDeAdjacencia, "Hercilio Luz", "Carajas")
print(imprime_menor_caminho(pai, "Hercilio Luz", "Carajas"))

['Hercilio Luz', 'Cataratas', 'Tancredo Neves', 'Carajas']


In [58]:
def dijkstra(graph,src,dest,visited=[],distances={},predecessors={}):
    """ calculates a shortest path tree routed in src
    """    
    # a few sanity checks
    if src not in graph:
        raise TypeError('The root of the shortest path tree cannot be found')
    if dest not in graph:
        raise TypeError('The target of the shortest path cannot be found')    
    # ending condition
    if src == dest:
        # We build the shortest path and display it
        path=[]
        pred=dest
        while pred != None:
            path.append(pred)
            pred=predecessors.get(pred,None)
        # reverses the array, to display the path nicely
        readable=path[0]
        for index in range(1,len(path)): readable = path[index]+'--->'+readable
        #prints it 
        print('shortest path - array: '+str(path))
        print("path: "+readable+",   cost="+str(distances[dest]))     
    else :     
        # if it is the initial  run, initializes the cost
        if not visited: 
            distances[src]=0
        # visit the neighbors
        for neighbor in graph[src] :
            if neighbor not in visited:
                new_distance = distances[src] + graph[src][neighbor]
                if new_distance < distances.get(neighbor,float('inf')):
                    distances[neighbor] = new_distance
                    predecessors[neighbor] = src
        # mark as visited
        visited.append(src)
        # now that all neighbors have been visited: recurse                         
        # select the non visited node with lowest distance 'x'
        # run Dijskstra with src='x'
        unvisited={}
        for k in graph:
            if k not in visited:
                unvisited[k] = distances.get(k,float('inf'))        
        x=min(unvisited, key=unvisited.get)
        dijkstra(graph,x,dest,visited,distances,predecessors)
        

In [59]:
dijkstra(listaDeAdjacencia,"Hercilio Luz", "Carajas")

shortest path - array: ['Carajas', 'Maraba', 'Presidente Juscelino Kubitschek', 'Viracopos', 'Hercilio Luz']
path: Hercilio Luz--->Viracopos--->Presidente Juscelino Kubitschek--->Maraba--->Carajas,   cost=2636.5130752005452


In [83]:
# create adjacency matrix for use in prims algorithm
# note: we could improve the running time of prims algorithm by 
# implementing a priority queue data structure instead of a matrix
def createAdjMatrix(V, G):
  
  adjMatrix = []
  
  # create N x N matrix filled with 0 edge weights between all vertices
  for i in range(0, V):
    adjMatrix.append([])
    for j in range(0, V):
      adjMatrix[i].append(0)
      
  # populate adjacency matrix with correct edge weights
  for i in range(0, len(G)):
    adjMatrix[G[i][0]][G[i][1]] = G[i][2]
    adjMatrix[G[i][1]][G[i][0]] = G[i][2]
    
  return adjMatrix

def prims(V, G):
  
  # create adj matrix from graph
  adjMatrix = createAdjMatrix(V, G)
  
  # arbitrarily choose initial vertex from graph
  vertex = 0
  
  # initialize empty edges array and empty MST
  MST = []
  edges = []
  visited = []
  minEdge = [None,None,float('inf')]
  
  # run prims algorithm until we create an MST
  # that contains every vertex from the graph
  while len(MST) != V-1:
    
    # mark this vertex as visited
    visited.append(vertex)
    
    # add each edge to list of potential edges
    for r in range(0, V):
      if adjMatrix[vertex][r] != 0:
        edges.append([vertex,r,adjMatrix[vertex][r]])
        
    # find edge with the smallest weight to a vertex
    # that has not yet been visited
    for e in range(0, len(edges)):
      if edges[e][2] < minEdge[2] and edges[e][1] not in visited:
        minEdge = edges[e]
        
    # remove min weight edge from list of edges
    edges.remove(minEdge)

    # push min edge to MST
    MST.append(minEdge)
      
    # start at new vertex and reset min edge
    vertex = minEdge[1]
    minEdge = [None,None,float('inf')]
    
  return MST
  
# graph vertices are actually represented as numbers
# like so: 0, 1, 2, ... V-1
a, b, c, d, e, f = 0, 1, 2, 3, 4, 5

# graph edges with weights
# diagram of graph is shown above
graph = [
  [a,b,2],
  [a,c,3],
  [b,d,3],
  [b,c,5],
  [b,e,4],
  [c,e,4],
  [d,e,2],
  [d,f,3],
  [e,f,5]
]

# pass the # of vertices and the graph to run prims algorithm 
print (prims(6, graph))

[[0, 1, 2], [0, 2, 3], [1, 3, 3], [3, 4, 2], [3, 5, 3]]
